In [16]:
from preprocessing.preprocessing import  get_preprocessed_brfss_train_test_split
from sklearn.ensemble import RandomForestClassifier
from IPython.core.display_functions import display
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_train_test_split()
cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
data_train.head(10)

,GenHealth,PhysHealth,MentHealth,Healthcare,MedCost,Checkup,HighBP,HighChol,HeartAttack,AngiCoro,Stroke,Asthma,Arthritis,Kidney,Sex,Income,SodiumSalt,Age,Height,Weight,BMI,Education,Alcohol,Smoking,FruitCons,VegetCons,PhysActivity,Muscles
434107,2.0,88.0,88.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8.0,0.0,9.0,152.0,5897.0,3.0,2.0,2.0,4.0,2.0,1.0,4.0,2.0
110299,4.0,88.0,88.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,4.0,2.0,13.0,175.0,7484.0,2.0,2.0,1.0,4.0,1.0,2.0,3.0,2.0
218193,3.0,88.0,88.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,77.0,0.0,8.0,180.0,8165.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,2.0
394642,2.0,88.0,1.0,1.0,1.0,2.0,3.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0,0.0,1.0,165.0,5443.0,2.0,2.0,1.0,4.0,1.0,1.0,1.0,2.0
354955,1.0,1.0,88.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,7.0,1.0,7.0,165.0,9480.0,4.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0
228337,1.0,88.0,88.0,1.0,2.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,6.0,0.0,3.0,168.0,8391.0,0.0,4.0,1.0,4.0,1.0,1.0,4.0,2.0
57536,2.0,88.0,3.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,8.0,0.0,9.0,165.0,6350.0,2.0,4.0,1.0,4.0,2.0,1.0,2.0,2.0
68009,3.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,7.0,0.0,11.0,165.0,6577.0,2.0,4.0,2.0,3.0,1.0,1.0,2.0,1.0
308419,3.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,6.0,0.0,11.0,155.0,6577.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,2.0
359773,1.0,5.0,3.0,1.0,2.0,1.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0,0.0,1.0,175.0,23088.0,4.0,2.0,2.0,3.0,1.0,1.0,3.0,1.0


In [5]:
random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(data_train, target_train)

accuracy = cross_val_score(random_forest_classifier, data_train, target_train, cv=cross_validation, scoring='accuracy')

for i, acc in enumerate(accuracy):
    print("Fold {}: Accuracy = {}%".format(i, acc * 100.0))

print("Average Accuracy = {}%".format(accuracy.mean() * 100.0))

Fold 0: Accuracy = 84.89919583191754%
Fold 1: Accuracy = 84.81991165477403%
Fold 2: Accuracy = 84.79725903273304%
Fold 3: Accuracy = 84.65284856722167%
Fold 4: Accuracy = 84.81708007701891%
Fold 5: Accuracy = 84.82840638803941%
Fold 6: Accuracy = 84.85389058783555%
Fold 7: Accuracy = 84.85105901008042%
Fold 8: Accuracy = 84.85389058783555%
Fold 9: Accuracy = 84.8109868327906%
Average Accuracy = 84.81845285702467%


### Grid search for random forest classifier
Unfortunately the grid search needs an extremely long time, so we decided to use a train/test split to find the best parameters

In [ ]:
random_forest_classifier = RandomForestClassifier()
parameters = {
    'max_depth': range(5, 10),
    'criterion': ["gini", "entropy", "log_loss"],
    'n_estimators': [100, 150, 200, 250, 300],
    'min_samples_split': range(2,5),
    'min_samples_leaf': range(1,5)
}

grid_search_estimator = GridSearchCV(random_forest_classifier, parameters, scoring='accuracy', cv=cross_validation, return_train_score=False)
grid_search_estimator.fit(data_train, target_train)

results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

This function will be used several times later to evaluate the model with the different parameters

In [9]:
def evaluate_random_forest_classifier(estimator: RandomForestClassifier):
    estimator.fit(data_train, target_train)
    predictions = estimator.predict(data_test)
    return accuracy_score(target_test, predictions)


### Parameter search
First we search for the best max depth

In [11]:
max_depth_parameters = range(2, 20)
for param in max_depth_parameters:
    estimator = RandomForestClassifier(max_depth=param)
    accuracy = evaluate_random_forest_classifier(estimator)
    print(f"Max depth {param} --> Accuracy: {accuracy}")

Max depth 2 --> Accuracy: 0.8429153924566769
Max depth 3 --> Accuracy: 0.8429153924566769
Max depth 4 --> Accuracy: 0.8431305923660664
Max depth 5 --> Accuracy: 0.8439800656926039
Max depth 6 --> Accuracy: 0.8448748442632235
Max depth 7 --> Accuracy: 0.846471854117114
Max depth 8 --> Accuracy: 0.8469928644240571
Max depth 9 --> Accuracy: 0.8475365273530411
Max depth 10 --> Accuracy: 0.8481708007701891
Max depth 11 --> Accuracy: 0.8483406954354966
Max depth 12 --> Accuracy: 0.8489296636085627
Max depth 13 --> Accuracy: 0.8488050741873372
Max depth 14 --> Accuracy: 0.8492468003171367
Max depth 15 --> Accuracy: 0.8491675161399932
Max depth 16 --> Accuracy: 0.8492354740061162
Max depth 17 --> Accuracy: 0.8493260844942803
Max depth 18 --> Accuracy: 0.8485558953448862
Max depth 19 --> Accuracy: 0.8486578321440706


In [12]:
criterion_parameter = ["gini", "entropy", "log_loss"]
for param in criterion_parameter:
    estimator = RandomForestClassifier(max_depth=2, criterion=param)
    accuracy = evaluate_random_forest_classifier(estimator)
    print(f"Criterion {param} --> Accuracy: {accuracy}")

Criterion gini --> Accuracy: 0.8429153924566769
Criterion entropy --> Accuracy: 0.8429153924566769
Criterion log_loss --> Accuracy: 0.8429153924566769


In [13]:
number_of_estimators_parameter = [100, 150, 200, 250, 300, 350, 400, 450, 500]
for param in number_of_estimators_parameter:
    estimator = RandomForestClassifier(max_depth=2, criterion="gini" ,n_estimators=param)
    accuracy = evaluate_random_forest_classifier(estimator)
    print(f"Number of estimators {param} --> Accuracy: {accuracy}")

Number of estimators 100 --> Accuracy: 0.8429153924566769
Number of estimators 150 --> Accuracy: 0.8429153924566769
Number of estimators 200 --> Accuracy: 0.8429153924566769
Number of estimators 250 --> Accuracy: 0.8429153924566769
Number of estimators 300 --> Accuracy: 0.8429153924566769
Number of estimators 350 --> Accuracy: 0.8429153924566769
Number of estimators 400 --> Accuracy: 0.8429153924566769
Number of estimators 450 --> Accuracy: 0.8429153924566769
Number of estimators 500 --> Accuracy: 0.8429153924566769


In [21]:
min_samples_split_parameter = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
for param in min_samples_split_parameter:
    estimator = RandomForestClassifier(max_depth=2, criterion="gini", n_estimators=100 ,min_samples_split=param)
    accuracy = evaluate_random_forest_classifier(estimator)
    print(f"Minimum samples for split {param} --> Accuracy: {accuracy}")

Minimum samples for split 0.05 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.1 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.15 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.2 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.25 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.3 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.35 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.4 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.45 --> Accuracy: 0.8429153924566769
Minimum samples for split 0.5 --> Accuracy: 0.8429153924566769


In [22]:
min_samples_leaf_parameter = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
for param in min_samples_leaf_parameter:
    estimator = RandomForestClassifier(max_depth=2, criterion="gini", n_estimators=100, min_samples_split= 0.3, min_samples_leaf=param)
    accuracy = evaluate_random_forest_classifier(estimator)
    print(f"Minimum samples for leaf {param} --> Accuracy: {accuracy}")

Minimum samples for leaf 0.05 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.1 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.15 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.2 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.25 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.3 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.35 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.4 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.45 --> Accuracy: 0.8429153924566769
Minimum samples for leaf 0.5 --> Accuracy: 0.8429153924566769
